In [ ]:

import numpy as np 
import pandas as pd

import matplotlib.pyplot as plt

import torch
from torch.utils.data import Dataset
from PIL import Image

# from skimage import io
# from torchvision.io import read_image


import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [ ]:
print(os.getcwd())  # Current Working Directory

In [ ]:
import zipfile

with zipfile.ZipFile("/kaggle/input/dogs-vs-cats/train.zip", "r") as zip_ref:
    zip_ref.extractall()

In [ ]:
with zipfile.ZipFile("/kaggle/input/dogs-vs-cats/test1.zip", "r") as zip_ref:
    zip_ref.extractall()

In [ ]:
train_dir = './train'
test_dir = './test1'

train_files = os.listdir(train_dir)
test_files = os.listdir(test_dir)

In [ ]:
sample_img = Image.open(os.path.join(train_dir, train_files[0]))
sample_img.show()

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
class CatsAndDogsDataset(Dataset):
    
    def __init__(self, img_dir, mode = 'train', transform = None):
        
        self.img_dir = img_dir
        self.transform = transform
        self.mode = mode
        
        self.img_names = os.listdir(img_dir)
        
    def __len__(self):
        
        return len(self.img_names)
    
    def __getitem__(self, index):
        
        filename = self.img_names[index]
        img = Image.open(os.path.join(self.img_dir, filename))
                         
        if self.transform:
            img = self.transform(img)
            
        img = img.numpy()
        
        if filename.split('.')[0] == 'cat':
            label = torch.tensor(0)
        else :
            label = torch.tensor(1)
            
        if self.mode == 'test' :
            
            label = filename
        
        return img.astype('float32'), label
                         
        
        
        

In [ ]:
# resize and rescale..

import torchvision.transforms as transforms

transform = transforms.Compose([
    
    transforms.Resize((224, 224)),
    transforms.CenterCrop((224, 224)),
    transforms.ToTensor(),
    
])

In [ ]:
dataset = CatsAndDogsDataset(train_dir, transform = transform)

In [ ]:
first_sample = dataset[0]
first_sample[0].shape

In [ ]:
first_sample[1]

In [ ]:
first_sample[0]

In [ ]:
import torchvision.models as models

model = models.resnet18(pretrained = True)



In [ ]:
for param in model.parameters():
    param.requires_grad = False

In [ ]:
import torch.nn as nn

num_ftrs = model.fc.in_features

model.fc = nn.Sequential(
    nn.Linear(num_ftrs, 50),
    nn.Linear(50, 2)
)

model = model.to(device)


In [ ]:
from torch.utils.data import random_split

train_size = int(0.8 * len(dataset))
val_size = len(dataset) - train_size
train_dataset, val_datset = random_split(dataset, [train_size, val_size])

In [ ]:
from torch.utils.data import DataLoader

train_dataloader = DataLoader(train_dataset, batch_size = 128, shuffle = True, num_workers = 4)

In [ ]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr = 0.01)

In [ ]:
num_epochs = 5

for epoch in range(num_epochs):
    
    running_loss = 0.0
    
    for i, data in enumerate(train_dataloader, 0):
        
        inputs, labels = data
        
        inputs, labels = inputs.to(device), labels.to(device)
        
        outputs = model(inputs)
        
        loss = criterion(outputs, labels)
        
        loss.backward()
        
        optimizer.step()
        
        optimizer.zero_grad()
        
        running_loss += loss.item()
        
    print(f'Epoch {epoch} : Loss = {running_loss:.3f}')
        
        

In [ ]:
test_transform = transforms.Compose([
    
    transforms.Resize((224, 224)),
    transforms.ToTensor()
])

In [ ]:
print(test_files[:5])

In [ ]:
test_dataset = CatsAndDogsDataset(test_dir, mode = 'test', transform = test_transform)
test_loader = DataLoader(test_dataset, batch_size = 128, shuffle = False, num_workers = 4)

In [ ]:
pred_list = []
ind_list = []

for x, fn in test_loader:
    
    with torch.no_grad():
        
        x = x.to(device)
        output = model(x)
        pred = torch.argmax(output, dim = 1)
        ind_list += [n[:-4] for n in fn]
        pred_list += [p.item() for p in pred]

In [ ]:
submission = pd.DataFrame({"id": ind_list, "label": pred_list})
submission.head()

In [ ]:
submission.to_csv('preds_resnet18v1.csv', index = False)